In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd

# Initialize the Chrome driver
driver = webdriver.Chrome()

# Open the website
url = "https://www.thegioididong.com/laptop"
driver.get(url)

# Allow some time for the page to load
time.sleep(3)

# Get page source and parse it with BeautifulSoup
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Close the driver after getting the source
driver.quit()

# Extract all product links from the main page
product_links = []
product_list = soup.find_all('a', class_='main-contain')

for link in product_list:
    href = link.get('href')
    if href:
        product_page_url = 'https://www.thegioididong.com' + href
        review_page_url = product_page_url + "/danh-gia"
        
        # Initialize the Chrome driver for product review page check
        driver = webdriver.Chrome()
        driver.get(review_page_url)
        time.sleep(3)
        
        # Check if review page exists
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        error_message = soup.find(text="Xin lỗi, chúng tôi không tìm thấy trang mà bạn cần!")
        
        if not error_message:
            product_links.append(review_page_url)
        
        driver.quit()

# Initialize lists to store product details
product_details = []
# Initialize the Chrome driver for product details
driver = webdriver.Chrome()
print(product_links)
for link in product_links:
    driver.get(link)
    time.sleep(3)
    
    # Get page source and parse it with BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Extract product name
    product_name = None
    product_name_tag = soup.find('div', class_='content')
    
    if product_name_tag:
        h1_tag = product_name_tag.find('h1')
        h2_tag = product_name_tag.find('h2')
        h3_tag = product_name_tag.find('h3')
        if h1_tag:
            product_name = h1_tag.text.strip()
        elif h2_tag:
            product_name = h2_tag.text.strip()
        elif h3_tag:
            product_name = h3_tag.text.strip()

    if not product_name:
        product_name = 'N/A'

    # Extract product price
    price_section = soup.find('div', class_='price')
    price_current = price_section.find('strong', class_='price-current').text.strip()
    price_old = price_section.find('p', class_='price-old').text.strip()
    price_percent = price_section.find('p', class_='price-percent').text.strip()

    # Extract and print reviews
    reviews = soup.find_all('li', class_='par')

    for review in reviews:
        name = review.find('p', class_='cmt-top-name').text.strip()
        rating = len(review.find_all('i', class_='iconcmt-starbuy'))
        comment = review.find('p', class_='cmt-txt').text.strip()
        time_used = review.find('span', class_='cmtd dot-line').text.strip()
        print(name)
        product_details.append({
            'Product Name': product_name,
            'Current Price': price_current,
            'Old Price': price_old,
            'Discount Percent': price_percent,
            'Reviewer': name,
            'Rating': rating,
            'Comment': comment,
            'Time Used': time_used
        })
        
driver.quit()


# Convert the collected data to a DataFrame
df = pd.DataFrame(product_details)

# Save the DataFrame to a CSV file
df.to_csv('product_reviews.csv', index=False, encoding='utf-8')

print("Data has been exported to 'product_reviews.csv'")
